# Derive TOA radiative fluxes for CERES SYN1deghourly data

This includes calculation of CRE, as well as changing signs and variable names to be consistent with the ICON-NWP model.

We do so for the observed and the computed initial fluxes. The latter are indicated in the CERES naming by 'ini\_'. When renaming to ICON-NWP variable names, we will add '\_obs' and '\_comp' to differentiate between the two. Note that the data quality summary of CERES SYN1deg Ed4A advices to compare all-sky TOA fluxes as a measure of the quality of all-sky fluxes, whereas clear-sky fluxes and CRE should only be taken from the computed initial fluxes. This is because the observed clear-sky fluxes are often set to the monthly climatology due to a lack of observed clear-sky scenes.

The data quality summary can be found here: https://ceres.larc.nasa.gov/documents/DQ_summaries/CERES_SYN1deg_Ed4A_DQS.pdf

Additional notes: 
* We assume that the CERES data was obtained previously and interpolated to a 1x1 deg lat-lon grid using cdo remapcon for a region covering the North Atlantic. 
* We use both the Terra-Aqua-Modis aas well as the Terra-NPP product. 
* The notebook is written for Mistral at DKRZ.

In [1]:
import numpy as np
import xarray as xr

In [2]:
def derive_and_store_toafluxes():

    for product in ['Terra-Aqua-MODIS_Ed4A','Terra-NPP_Ed1A']:
        ds = ( xr.open_mfdataset('/work/bb1018/b380459/OBS/CERES/CERES_SYN1deg/'+
                                 'CERES_SYN1deg-1H_'+product+'_Subset_201609-201610.gridr1x1.nc', 
                                 combine='by_coords', parallel=True, chunks={'time': 1})
                                 [['toa_lw_clr_1h', 'toa_lw_all_1h',
                                   'toa_sw_clr_1h', 'toa_sw_all_1h', 'toa_solar_all_1h', 
                                   'ini_toa_lw_clr_1h', 'ini_toa_lw_all_1h',
                                   'ini_toa_sw_clr_1h', 'ini_toa_sw_all_1h', 
                                 ]] )
    
        #  derive TOA radiative fluxes 
        ds['swtoaclr_obs'] = ds['toa_solar_all_1h' ] - ds['toa_sw_clr_1h'    ]
        ds['swtoaclr_com'] = ds['toa_solar_all_1h' ] - ds['ini_toa_sw_clr_1h']
        ds['swtoacre_obs'] = ds['toa_sw_clr_1h'    ] - ds['toa_sw_all_1h'    ]
        ds['swtoacre_com'] = ds['ini_toa_sw_clr_1h'] - ds['ini_toa_sw_all_1h']
        ds['lwtoaclr_obs'] = ds['toa_lw_clr_1h' ]*(-1)
        ds['lwtoaclr_com'] = ds['ini_toa_lw_clr_1h']*(-1)
        ds['lwtoacre_obs'] = ds['toa_lw_clr_1h'    ] - ds['toa_lw_all_1h'    ]
        ds['lwtoacre_com'] = ds['ini_toa_lw_clr_1h'] - ds['ini_toa_lw_all_1h']
        
        # rename all-sky ceres variables to icon names
        ds = ds.rename({'toa_solar_all_1h' : 'sod_t',
                        'toa_sw_all_1h'    : 'sou_t_obs', 
                        'toa_lw_all_1h'    : 'thb_t_obs',
                        'ini_toa_sw_all_1h': 'sou_t_com', 
                        'ini_toa_lw_all_1h': 'thb_t_com',
                       })
        
        # for longwave fluxes adjust signs to be consistent with ICON
        ds['thb_t_obs'  ] = -1*ds['thb_t_obs']
        ds['thb_t_com'  ] = -1*ds['thb_t_com']
        
        # only keep fluxes consistent with ICON-NWP
        ds = ds[['sod_t', 'sou_t_obs', 'thb_t_obs', 'sou_t_com', 'thb_t_com',
                 'swtoaclr_obs', 'swtoaclr_com', 'swtoacre_obs', 'swtoacre_com',
                 'lwtoaclr_obs', 'lwtoaclr_com', 'lwtoacre_obs', 'lwtoacre_com']]
        
        ds.attrs['CERES-product'] = 'CERES_SYN1deg-1H_'+product
    
        ds.to_netcdf('/work/bb1018/b380459/OBS/CERES/CERES_SYN1deg/CERES_SYN1deg-1H_'+product+'_Subset_201609-201610.gridr1x1.derivedTOAfluxes.nc')

derive_and_store_toafluxes()

Manual checking of the derived fluxes by means of ds.mean.compute() verified that the calculation was done correctly: the signs are correct and the values are meaningful.